In [1]:
##funkcije

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, confusion_matrix, plot_confusion_matrix, roc_curve, auc

Korisni termini:
<ul>
    <li><em>True positives (TP)</em>: transakcije koje su prevare i naš model ih je ispravno klasificirao (kao prevare)</li>
    <li><em>False positives (FP)</em>: transakcije koje su valjane, ali naš model ih je pogrešno klasificirao (kao prevare)</li>
    <li><em>True negatives (TN)</em>: transakcije koje su valjane i naš model ih je ispravno klasificirao (kao valjane)</li>
    <li><em>False negatives (FN)</em>: transakcije koje su prevare, ali naš model ih je pogrešno klasificirao (kao valjane)</li> <br>
    <li><em>Precision (hrv. preciznost, oznaka: <i>P</i>)</em> -  omjer transakcija koje je model ispravno klasificirao kao prevare i svih transakcija koje je klasificirao kao prevare. Formulom: $P = \frac{TP}{TP + FP}$</li>
    <li><em>Recall (hrv. osjetljivost, oznaka: <i>R</i>)</em> -  omjer transakcija koje je model ispravno klasificirao kao prevare i svih transakcija koje ustvari jesu prevare. Formulom: $R = \frac{TP}{TP + FN}$</li>
    <li><em>F1-score(hrv. f1-mjera, oznaka: <i>F1</i>)</em> - mjera koja povezuje preciznost i osjetljivost formulom: $F1 = \frac{2PR}{P + R}$</li>
</ul>



Sljedeću funkciju koristit ćemo da bi ispisali preciznost, osjetljivost i f1-mjeru. Primjetimo da ih ne trebamo računati ručno po formulama koje smo naveli, već ćemo samo koristiti funkcije implementirane u biblioteci <i>sklearn.metrics</i>. 

In [10]:
def precision_recall_f1_scores(correct, predicted):
    print('\tPrecision score: ' + str(round(precision_score(correct, predicted), 4) * 100) + '%')
    print('\tRecall score: ' + str(round(recall_score(correct, predicted), 4) * 100) + '%')
    print('\tF1 score: ' + str(round(f1_score(correct, predicted), 4) * 100) + '%')

Također da bi bolje procijenili 'kvalitetu' našeg modela crtamo i konfuzijsku matricu (odnosno 2). Lijeva konfuzijska matrica će nam služiti da bi izvukli čiste brojčane vrijednosti (primjerice koliko smo prevara ispravno klasificirali - dolje desno), dok ćemo desnu konfuzijsku matricu koristiti da bi dobili relativne vrijednosti (primjerice postotak ispravno klasificiranih prevara - isto dolje desno). 

In [11]:
def plot_confusion_matrix(conf_mat, y_test_non_fraud, y_test_fraud):
    fig, ax = plt.subplots(1,2,figsize=(15,6))
    labels = ['Valjana', 'Prevara']
    ax[0].set_title('Konfuzijska matrica')
    sns.heatmap(conf_mat, ax=ax[0], linewidths=0.5, annot=True,
            square=True, cmap = plt.cm.coolwarm, linecolor='white',
               xticklabels=labels, yticklabels=labels)
    
    
    ax[1].set_title('Normalizirana konfuzijska matrica')
    norm_coef = np.array([[1/y_test_non_fraud, 1/y_test_non_fraud],
                         [1/y_test_fraud, 1/y_test_fraud]])
    normalized_cm = conf_mat * norm_coef 
    sns.heatmap(normalized_cm, ax=ax[1], linewidths=0.5, annot=True,
            square=True, cmap = plt.cm.coolwarm, linecolor='white',
               xticklabels=labels, yticklabels=labels)
    
    plt.show()


Kao zadnju metriku ćemo koristiti roc krivulju te PR krivulju. Doduše njih možemo nacrtati samo za logističku regresiju. 

In [12]:
def plot_roc_auc_curve(correct, predicted):
    fpr, tpr, threshold = roc_curve(correct, predicted)
    area_under_curve = auc(fpr,tpr)
    plt.plot([0, 1], [0, 1], "k--")
    plt.plot(fpr, tpr, label="AUC = " + str(round(area_under_curve,3)))
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.title("ROC curve")
    plt.legend()
    plt.show()

In [13]:
def plot_precision_recall_curve(correct, predicted):
    precision, recall, threshold = precision_recall_curve(correct, predicted)
    aupr = auc(recall, precision)
    plt.step(recall, precision, color = 'b', alpha = 0.2,
             where = 'post')
    plt.fill_between(recall, precision, step ='post', alpha = 0.2,
                 color = 'b')

    plt.plot(recall, precision, linewidth=2, label="AUC = " + str(round(aupr,3)))
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision Recall Curve')
    plt.legend()
    plt.show();


Još ćemo dodati funkciju <i>print_everything</i> koja nam omogućuje da u jednom retku ispišemo sve ranije navedene funkcije.

In [16]:
def print_everything(y_data_test, full_pred, non_frauds, frauds, full_pred_score, log_reg_bool):
    print('-'*100)
    print('Predikcija za cijeli dataset:')
    precision_recall_f1_scores(y_data_test, full_pred)
    print()
    print('-'*100)
    print(classification_report(y_data_test, full_pred))
    print()
    print('-'*100)
    plot_confusion_matrix(confusion_matrix(y_data_test, full_pred), y_data_test_non_fraud_num, y_data_test_fraud_num)
    print('-'*100)
    
    if log_reg_bool:
        plot_roc_auc_curve(y_data_test, full_pred_score)
        plot_precision_recall_curve(y_data_test, full_pred_score)